## Load Files and Vectorize 

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [4]:
with open(r"data/df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [3]:
with open(r"data/df_best_category.pkl", "rb") as input_file:
    df_best_category = pickle.load(input_file)

In [5]:
with open (r"data/dictionary.pkl", "rb") as input_file: 
    dictionary = pickle.load(input_file)

In [29]:
with open (r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model_1 = pickle.load(input_file)

## Third Model 

In [33]:
import textstat
import operator
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def give_harder_level(text):
    """
    Takes in text and returns a harder level read
    """
    unseen_document = str(text)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    text_topics = lda_model_1[bow_vector]
    print(text_topics)
    
    ##find all the articles that are easier than this article 
    all_harder_text = []
    input_score = textstat.flesch_kincaid_grade(text)
    for i in range(len(df_corpus2['score'])):
        if df_corpus2['score'][i] > input_score:
            all_harder_text.append(df_corpus2['content'][i])
    
    vec = TfidfVectorizer(stop_words='english', max_features=2000)
    vec = vec.fit(all_harder_text)
    corpus2_vectors = vec.transform(all_harder_text).toarray()
    user_doc = text
    user_doc_vector = vec.transform([user_doc]).toarray() 
    user_doc_vector_short = np.argsort(user_doc_vector)[-25:]
    distances = cdist(user_doc_vector_short,
                  corpus2_vectors[-25:],
                  metric='cosine')[0]
    ranking = np.argsort(distances)
    top = ranking[0]
    best_match = df_corpus2['content'][top]
        
    print(distances[top])
    return (best_match)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherzyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
sample = "On one level, this is a business dispute that highlights tensions in the music industry (both intractably ancient and very current). When Swift signed her six-album deal that furnished Big Machine with rights to her masters, she was partaking in a classic arrangement for new artists: handing over future control of music in exchange for start-up promotional, recording, and distribution help. As she’s risen to megastardom, she’s chafed at that arrangement the same way that many successful musicians have chafed at not having ownership of their work. Prince, who famously protested Warner Bros. Records in 1993 by writing the word slave on his cheek, struck a deal for his own back catalog at great cost in 2014. Paul McCartney preached for years about the importance of artists owning music—and one person he preached to, Michael Jackson, ended up buying the rights to the Beatles’ catalog from under him. West himself recently filed suit to gain control of his own masters."

In [35]:
give_harder_level(sample)

[(6, 0.12670091), (9, 0.8628927)]
0.6822204939076502


'Value or values may refer to:\n\nValue (philosophy),\nValue (ethics) it may be described as treating actions themselves as abstract objects, putting value to them\nValues (Western philosophy) expands the notion of value beyond that of ethics, but limited to Western sources\nSocial imaginary is the set of values, institutions, laws, and symbols common to a particular social group\nValue (economics), a measure of the benefit that may be gained from goods or service\nTheory of value (economics), the study of the concept of economic value\nValue (marketing), the difference between a customer\'s evaluation of benefits and costs\nValue investing, an investment paradigm\n\nOther uses\nValue, also known as lightness or tone, a representation of variation in the perception of a color or color space\'s brightness\nValue (computer science), an expression that implies no (further) (mathematical) processing; a "normal form"\nValue (mathematics), a property such as number assigned to or calculated 